In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
driver = webdriver.Chrome('C:/Users/jeann/Documents/playwithdata/chromedriver.exe')
url = 'http://search.danawa.com/dsearch.php?query=홍삼&volumeType=allvs&page=1&limit=40&sort=saveDESC&list=list&boost=true&addDelivery=N&attributeOption=2276|35496|228139|OR&recommendedSort=Y&defaultUICategoryCode=1622276&defaultPhysicsCategoryCode=46803%7C46817%7C46824%7C0&defaultVmTab=336&defaultVaTab=0&tab=goods'
driver.get(url)

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

In [ ]:
prod_items = soup.select('ul.product_list > li.prod_item')
len(prod_items)

In [ ]:
net_items = soup.select("li[id^='productItem']")
len(net_items)

In [ ]:
title = net_items[0].select('p.prod_name > a')[0].text.strip()
print(title)

In [ ]:
spec_list = net_items[0].select('div.spec_list')[0].text.strip()
print(spec_list)

In [ ]:
price = net_items[0].select('div.prod_pricelist > ul > li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",","")
print(price)

In [ ]:
product_data = []

for item in net_items:
    try:
        title = item.select('p.prod_name > a')[0].text.strip()
    except:
        title = ''
    try:
        spec_list = item.select('div.spec_list')[0].text.strip()
    except:
        spec_list =''
    try:
        price = item.select('div.prod_pricelist > ul > li.rank_one > p > a > strong')[0].text.strip().replace(",","")
    except:
        price = 0
    try:
        unit_price = item.select('div.prod_pricelist > ul > li.rank_one > p.memory_sect > a > span.memory_price_sect > em.lowest')[0].text.strip().replace(",","")
    except:
        unit_price = 0
    
    product_data.append([title,spec_list,price,unit_price])

print(len(product_data))

In [ ]:
 def get_items(net_items):
    product_data = []

    for item in net_items:
        try:
            title = item.select('p.prod_name > a')[0].text.strip()
        except:
            title = ''
        try:
            spec_list = item.select('div.spec_list')[0].text.strip()
        except:
            spec_list =''
        try:
            price = item.select('div.prod_pricelist > ul > li.rank_one > p > a > strong')[0].text.strip().replace(",","")
        except:
            price = 0
        try:
            unit_price = item.select('div.prod_pricelist > ul > li.rank_one > p.memory_sect > a > span.memory_price_sect > em.lowest')[0].text.strip().replace(",","")
        except:
            unit_price = 0

        product_data.append([title,spec_list,price,unit_price])
        
    return product_data

In [ ]:
net_items = soup.select("li[id^='productItem']")
product_data = get_items(net_items)
print(len(product_data))

In [ ]:
def get_search_page_url(keyword, page):
    return 'http://search.danawa.com/dsearch.php?query={}&volumeType=allvs&page={}&limit=40&sort=saveDESC&list=list&boost=true&addDelivery=N&attributeOption=2276|35496|228139|OR&recommendedSort=Y&defaultUICategoryCode=1622276&defaultPhysicsCategoryCode=46803%7C46817%7C46824%7C0&defaultVmTab=336&defaultVaTab=0&tab=goods'.format(keyword,page)

keyword = '홍삼'
page = 1
url = get_search_page_url(keyword, page)
print(url)

In [ ]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

driver = webdriver.Chrome('C:/Users/jeann/Documents/playwithdata/chromedriver.exe')

driver.implicitly_wait(3)

keyword = '홍삼'
total_page = 9
prod_data_total = []

for page in tqdm(range(1,total_page +1)):
    url = get_search_page_url(keyword,page)
    driver.get(url)
    time.sleep(5)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    
    net_items = soup.select("li[id^='productItem']")
    item_list = get_items(net_items)
    
    prod_data_total += item_list

In [ ]:
import pandas as pd
data = pd.DataFrame(prod_data_total)
data.columns = ['상품명','스펙목록','가격','단가']
data.to_excel('./files/1_danawa_crawling_result.xlsx', index = False)

In [ ]:
data = pd.read_excel('./files/1_danawa_crawling_result.xlsx')
data.info()
data.head()

In [ ]:
# 10g당 단가를 10g당 진세노사이드 함량으로 나눠서 진세노사이드 1mg 당 단가를 추출하기 위한 전처리 (10g당 단가/10g당 진세노사이드 함량)
# 수집시점 기준으로 총 335개의 리스트 추출됨
ginseno_portion = []
unit_price = []
  

for cost in data['단가']:        
    if cost !=0:
        price = int(cost.split('원')[0])
    elif cost ==0:
        price = 0
    
    unit_price.append(price)
    

for detail in data['스펙목록']:
    spec_list = detail.split('/')

    for spec in spec_list:
        if '진세노사이드:' in spec:
            ginseno = spec
            ginseno_amount = ginseno.split(':')[1]
            ginseno_per1g = float(ginseno_amount.split('mg')[0])
            ginseno_per10g = ginseno_per1g *10

    ginseno_portion.append(ginseno_per10g)

    
print("10g당 진세노함량: ",len(ginseno_portion), ginseno_portion[0:10])
print("10g당 가격: ", len(unit_price),unit_price[0:10])

In [ ]:
#홍삼농축액 함량만 따로 빼서 새로운 컬럼으로 구성하고, 이중에서 100%인 행만 추출하고자 함
concentrate =[]

for spec_data in data['스펙목록']:
    spec_list = spec_data.split('/')
    
    percentile = None
    
    for spec in spec_list:
        if '홍삼농축액' in spec:
            percent = spec.split('홍삼농축액')[1]
            percentile = percent.split('%')[0].strip()
            
    concentrate.append(percentile)

In [ ]:
# 새로운 데이터 프레임으로 다시 구성 (홍삼농축액 함량, 10g당 진세노함량, 10g당 가격 정보를 추가로 생성함)
pd_data = pd.DataFrame()
pd_data['상품명'] = data['상품명']
pd_data['스펙목록'] = data['스펙목록']
pd_data['홍삼농축액 함량'] = concentrate
pd_data['가격'] = data['가격']
pd_data['10g당 진세노함량'] = ginseno_portion
pd_data['10g당 가격'] = unit_price
pd_data.head()

In [ ]:
# 10g당 단가를 10g당 진세노사이드 함량으로 나눠서 진세노사이드 1mg 당 단가 추출작업 (10g당 단가/10g당 진세노사이드 함량)
final_unit_price =[]

for i in range(len(pd_data)):
    unit_cost = pd_data['10g당 가격'][i]/pd_data['10g당 진세노함량'][i]
    
    final_unit_price.append(unit_cost)

print("진세노 1mg당 가격: ", len(final_unit_price),final_unit_price[0:10])

In [ ]:
#진세노사이드 1mg당 단가 자료에 해당하는 컬럼을 추가 (데이터가 제대로 나왔는지 확인하기 위해 draft 파일로 저장함)
pd_data['진세노 1mg당 가격'] = final_unit_price
pd_data.to_excel('./files/redginseng_draft.xlsx', index = False)

In [ ]:
#홍삼농축액 함량 100%인 제품만 비교하기 위해 홍삼농축액 함량 100%인 제품만 필터링을 걸고, 이를 오름차순으로 정리함
condition = pd_data['홍삼농축액 함량'].isin(['100'])
condition2 = (pd_data['10g당 가격']==0)
data1 = pd_data[condition]
data2= data1[condition2 == False]
final_list = data2.sort_values(['진세노 1mg당 가격'],ascending=True)
final_list.head(10)

In [ ]:
#홍삼 농축액 100%인 제품은 총 78개인 것으로 확인됨
print(len(final_list))

In [ ]:
final_list.to_excel('./files/redginseng_final.xlsx', index = False)